<a href="https://colab.research.google.com/github/aptr288/Claim_prediction/blob/master/Claim_Prediction_Codebase.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pet Insurance claim prediction.



Libraries downloaded and imported. 


In [1]:
!pip install pandas

In [0]:
import pandas as pd
import numpy as np
import datetime
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import  svm,  tree
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.tree import DecisionTreeRegressor 
from sklearn import metrics
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
import xgboost as xgb

Mounting Google drive to work with files uploaded to Gdrive

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import os
os.chdir("/content/drive/My Drive/Colab Notebooks/Trupanion/")

In [5]:
!ls

claimdata.csv			 Code_Trupanian.ipynb	       petdata.csv
Claim_Prediction_Codebase.ipynb  Pet_claim_predicted_july.csv


Loading Pet data into dataframe

In [6]:
pet_data_df = pd.read_csv('petdata.csv', delimiter=',', header = 'infer')
pet_data_df.head(5)

,PetId,EnrollDate,CancelDate,Species,Breed,AgeAtEnroll
0,0,2015-08-11,2018-07-13,Dog,Mixed Breed,5 years old
1,1,2014-02-15,2016-01-25,Cat,Mixed Breed,3 years old
2,2,2014-12-20,NaN,Dog,French Bulldog,8 weeks to 12 months old
3,3,2015-01-09,2015-01-14,Dog,Mixed Breed,1 year old
4,4,2014-07-20,NaN,Cat,Mixed Breed,8 weeks to 12 months old


Loading Claim data into dataframe

In [7]:
claim_data_df = pd.read_csv('claimdata.csv', delimiter=',', header= 'infer')
claim_data_df.head(5)

,ClaimId,PetId,ClaimDate,ClaimAmount
0,0,9273,2014-02-25,389.65
1,1,1988,2014-03-10,455.40
2,2,2452,2014-04-02,1953.79
3,3,3658,2014-04-05,243.74
4,5,9147,2014-03-18,206.42


In [0]:
pet_data_df['EnrollDate'] =  pd.to_datetime(pet_data_df['EnrollDate'], format='%Y-%m-%d')
pet_data_df['CancelDate'] =  pd.to_datetime(pet_data_df['CancelDate'], format='%Y-%m-%d')
claim_data_df['ClaimDate'] =  pd.to_datetime(claim_data_df['ClaimDate'], format='%Y-%m-%d')

**Joining pet data with claim, left joint to get each claim per pet**

In [9]:
claim_data_per_pet = pd.merge(pet_data_df, claim_data_df, how='left' , on = 'PetId')
claim_data_per_pet.head(20)

,PetId,EnrollDate,CancelDate,Species,Breed,AgeAtEnroll,ClaimId,ClaimDate,ClaimAmount
0,0,2015-08-11,2018-07-13,Dog,Mixed Breed,5 years old,NaN,NaT,NaN
1,1,2014-02-15,2016-01-25,Cat,Mixed Breed,3 years old,NaN,NaT,NaN
2,2,2014-12-20,NaT,Dog,French Bulldog,8 weeks to 12 months old,7633.0,2018-02-14,82.80
3,2,2014-12-20,NaT,Dog,French Bulldog,8 weeks to 12 months old,13987.0,2018-08-07,190.69
4,2,2014-12-20,NaT,Dog,French Bulldog,8 weeks to 12 months old,24729.0,2018-04-14,346.61
5,2,2014-12-20,NaT,Dog,French Bulldog,8 weeks to 12 months old,25317.0,2018-07-14,27.25
6,2,2014-12-20,NaT,Dog,French Bulldog,8 weeks to 12 months old,26643.0,2018-11-16,186.76
7,2,2014-12-20,NaT,Dog,French Bulldog,8 weeks to 12 months old,27391.0,2018-06-15,162.01
8,2,2014-12-20,NaT,Dog,French Bulldog,8 weeks to 12 months old,29542.0,2019-03-25,191.44
9,2,2014-12-20,NaT,Dog,French Bulldog,8 weeks to 12 months old,35612.0,2015-12-30,75.32


In [10]:
claim_data_per_pet['ClaimAmount'].fillna(0, inplace=True)
claim_data_per_pet.head()

,PetId,EnrollDate,CancelDate,Species,Breed,AgeAtEnroll,ClaimId,ClaimDate,ClaimAmount
0,0,2015-08-11,2018-07-13,Dog,Mixed Breed,5 years old,NaN,NaT,0.00
1,1,2014-02-15,2016-01-25,Cat,Mixed Breed,3 years old,NaN,NaT,0.00
2,2,2014-12-20,NaT,Dog,French Bulldog,8 weeks to 12 months old,7633.0,2018-02-14,82.80
3,2,2014-12-20,NaT,Dog,French Bulldog,8 weeks to 12 months old,13987.0,2018-08-07,190.69
4,2,2014-12-20,NaT,Dog,French Bulldog,8 weeks to 12 months old,24729.0,2018-04-14,346.61


# Feature Engineering 

In [11]:
claim_data_per_pet['ClaimYear'] = pd.DatetimeIndex(claim_data_per_pet['ClaimDate']).year.fillna(0)
claim_data_per_pet['ClaimMonth'] = pd.DatetimeIndex(claim_data_per_pet['ClaimDate']).month.fillna(0)
claim_data_per_pet['ClaimlDay'] = pd.DatetimeIndex(claim_data_per_pet['ClaimDate']).day.fillna(0)
claim_data_per_pet.head()

,PetId,EnrollDate,CancelDate,Species,Breed,AgeAtEnroll,ClaimId,ClaimDate,ClaimAmount,ClaimYear,ClaimMonth,ClaimlDay
0,0,2015-08-11,2018-07-13,Dog,Mixed Breed,5 years old,NaN,NaT,0.00,0.0,0.0,0.0
1,1,2014-02-15,2016-01-25,Cat,Mixed Breed,3 years old,NaN,NaT,0.00,0.0,0.0,0.0
2,2,2014-12-20,NaT,Dog,French Bulldog,8 weeks to 12 months old,7633.0,2018-02-14,82.80,2018.0,2.0,14.0
3,2,2014-12-20,NaT,Dog,French Bulldog,8 weeks to 12 months old,13987.0,2018-08-07,190.69,2018.0,8.0,7.0
4,2,2014-12-20,NaT,Dog,French Bulldog,8 weeks to 12 months old,24729.0,2018-04-14,346.61,2018.0,4.0,14.0


In [12]:
claim_data_per_pet_month = claim_data_per_pet.groupby(['PetId','ClaimYear','ClaimMonth'], as_index=False)[['ClaimAmount']].sum()
claim_data_per_pet_month.describe()

,PetId,ClaimYear,ClaimMonth,ClaimAmount
count,28673.000000,28673.000000,28673.000000,28673.000000
mean,4943.241691,1596.144282,5.040073,525.899504
std,2853.616239,818.774254,3.985539,1381.544131
min,0.000000,0.000000,0.000000,0.000000
25%,2579.000000,2012.000000,1.000000,30.760000
50%,4882.000000,2016.000000,5.000000,138.600000
75%,7338.000000,2018.000000,8.000000,425.650000
max,9999.000000,2019.000000,12.000000,48691.460000


Total amount of claims done for each pet

In [0]:
def get_claims_count(ClaimsCount,claim_date):
  if (claim_date is pd.NaT):
    return 0
  else:
    return ClaimsCount
#df.groupby('PetId')[['ClaimAmount']].count()

In [14]:
total_claim_amount_per_petid = claim_data_per_pet.groupby('PetId')[['ClaimAmount']].sum()
total_claim_amount_per_petid = total_claim_amount_per_petid.rename(columns = {"PetId": "PetId",  "ClaimAmount": "TotalClaimedAmount"})

total_num_cliams_per_petid = claim_data_per_pet.groupby('PetId')[['ClaimAmount']].count()
total_num_cliams_per_petid = total_num_cliams_per_petid.rename(columns = {"PetId": "PetId",  "ClaimAmount": "ClaimsCountIntermediate" })
extracted_features = pd.merge(total_claim_amount_per_petid, total_num_cliams_per_petid, how='inner' , on = 'PetId')
claim_data_per_pet = pd.merge(claim_data_per_pet, extracted_features, how='left' , on = 'PetId')
claim_data_per_pet['ClaimsCountRevised'] = claim_data_per_pet.apply(lambda x: get_claims_count(x.ClaimsCountIntermediate, x.ClaimDate), axis=1)
claim_data_per_pet.head()

,PetId,EnrollDate,CancelDate,Species,Breed,AgeAtEnroll,ClaimId,ClaimDate,ClaimAmount,ClaimYear,ClaimMonth,ClaimlDay,TotalClaimedAmount,ClaimsCountIntermediate,ClaimsCountRevised
0,0,2015-08-11,2018-07-13,Dog,Mixed Breed,5 years old,NaN,NaT,0.00,0.0,0.0,0.0,0.00,1,0
1,1,2014-02-15,2016-01-25,Cat,Mixed Breed,3 years old,NaN,NaT,0.00,0.0,0.0,0.0,0.00,1,0
2,2,2014-12-20,NaT,Dog,French Bulldog,8 weeks to 12 months old,7633.0,2018-02-14,82.80,2018.0,2.0,14.0,2643.44,14,14
3,2,2014-12-20,NaT,Dog,French Bulldog,8 weeks to 12 months old,13987.0,2018-08-07,190.69,2018.0,8.0,7.0,2643.44,14,14
4,2,2014-12-20,NaT,Dog,French Bulldog,8 weeks to 12 months old,24729.0,2018-04-14,346.61,2018.0,4.0,14.0,2643.44,14,14


In [15]:
claims_count_revised = claim_data_per_pet.loc[:, ('PetId', 'ClaimsCountRevised')] #claim_data_per_pet.loc([['PetId', 'ClaimsCountRevised']])
claims_count_revised.drop_duplicates(subset ="PetId", 
                     keep = "first", inplace = True)
extracted_features = pd.merge(extracted_features, claims_count_revised, how='inner' , on = 'PetId')
extracted_features.head()

,PetId,TotalClaimedAmount,ClaimsCountIntermediate,ClaimsCountRevised
0,0,0.00,1,0
1,1,0.00,1,0
2,2,2643.44,14,14
3,3,0.00,1,0
4,4,1356.79,1,1


In [0]:
extracted_features['InsuranceCancelled'] = pet_data_df['CancelDate'].apply(lambda x: 1 if x is pd.NaT  else 0)

In [0]:
def get_insured_period(enrolled,cancelled):
  if (cancelled is pd.NaT):
    return abs(enrolled - pd.Timestamp('2019-07-01')).days
  else:
    return abs(enrolled - cancelled).days

In [18]:
#extracted_features['InsurancedPeriod'] =  abs(pet_data_df['EnrollDate'] - pet_data_df['CancelDate'])
extracted_features['InsurancedPeriod'] = pet_data_df.apply(lambda x: get_insured_period(x.EnrollDate, x.CancelDate), axis=1)

extracted_features['ClaimFreqPerMonth'] = (extracted_features['ClaimsCountRevised']*30)/extracted_features['InsurancedPeriod']
extracted_features.head(20)

,PetId,TotalClaimedAmount,ClaimsCountIntermediate,ClaimsCountRevised,InsuranceCancelled,InsurancedPeriod,ClaimFreqPerMonth
0,0,0.00,1,0,0,1067,0.000000
1,1,0.00,1,0,0,709,0.000000
2,2,2643.44,14,14,1,1654,0.253930
3,3,0.00,1,0,0,5,0.000000
4,4,1356.79,1,1,1,1807,0.016602
5,5,734.73,1,1,0,978,0.030675
6,6,0.00,1,0,0,951,0.000000
7,7,0.00,1,0,1,630,0.000000
8,8,0.00,1,0,1,1849,0.000000
9,9,0.00,1,0,0,212,0.000000


In [19]:
claim_data_per_pet_month = pd.merge(claim_data_per_pet_month, extracted_features, how='left' , on = 'PetId')
claim_data_per_pet_month.head(20)

,PetId,ClaimYear,ClaimMonth,ClaimAmount,TotalClaimedAmount,ClaimsCountIntermediate,ClaimsCountRevised,InsuranceCancelled,InsurancedPeriod,ClaimFreqPerMonth
0,0,0.0,0.0,0.00,0.00,1,0,0,1067,0.000000
1,1,0.0,0.0,0.00,0.00,1,0,0,709,0.000000
2,2,2015.0,12.0,75.32,2643.44,14,14,1,1654,0.253930
3,2,2016.0,1.0,61.06,2643.44,14,14,1,1654,0.253930
4,2,2017.0,6.0,63.25,2643.44,14,14,1,1654,0.253930
5,2,2017.0,7.0,144.12,2643.44,14,14,1,1654,0.253930
6,2,2017.0,11.0,265.93,2643.44,14,14,1,1654,0.253930
7,2,2017.0,12.0,567.32,2643.44,14,14,1,1654,0.253930
8,2,2018.0,2.0,82.80,2643.44,14,14,1,1654,0.253930
9,2,2018.0,4.0,346.61,2643.44,14,14,1,1654,0.253930


In [20]:
claim_data_per_pet_month = pd.merge(claim_data_per_pet_month, pet_data_df, how='left' , on = 'PetId')
claim_data_per_pet_month.head(20)

,PetId,ClaimYear,ClaimMonth,ClaimAmount,TotalClaimedAmount,ClaimsCountIntermediate,ClaimsCountRevised,InsuranceCancelled,InsurancedPeriod,ClaimFreqPerMonth,EnrollDate,CancelDate,Species,Breed,AgeAtEnroll
0,0,0.0,0.0,0.00,0.00,1,0,0,1067,0.000000,2015-08-11,2018-07-13,Dog,Mixed Breed,5 years old
1,1,0.0,0.0,0.00,0.00,1,0,0,709,0.000000,2014-02-15,2016-01-25,Cat,Mixed Breed,3 years old
2,2,2015.0,12.0,75.32,2643.44,14,14,1,1654,0.253930,2014-12-20,NaT,Dog,French Bulldog,8 weeks to 12 months old
3,2,2016.0,1.0,61.06,2643.44,14,14,1,1654,0.253930,2014-12-20,NaT,Dog,French Bulldog,8 weeks to 12 months old
4,2,2017.0,6.0,63.25,2643.44,14,14,1,1654,0.253930,2014-12-20,NaT,Dog,French Bulldog,8 weeks to 12 months old
5,2,2017.0,7.0,144.12,2643.44,14,14,1,1654,0.253930,2014-12-20,NaT,Dog,French Bulldog,8 weeks to 12 months old
6,2,2017.0,11.0,265.93,2643.44,14,14,1,1654,0.253930,2014-12-20,NaT,Dog,French Bulldog,8 weeks to 12 months old
7,2,2017.0,12.0,567.32,2643.44,14,14,1,1654,0.253930,2014-12-20,NaT,Dog,French Bulldog,8 weeks to 12 months old
8,2,2018.0,2.0,82.80,2643.44,14,14,1,1654,0.253930,2014-12-20,NaT,Dog,French Bulldog,8 weeks to 12 months old
9,2,2018.0,4.0,346.61,2643.44,14,14,1,1654,0.253930,2014-12-20,NaT,Dog,French Bulldog,8 weeks to 12 months old


In [21]:
claim_data_per_pet_month['EnrollYear'] = pd.DatetimeIndex(claim_data_per_pet_month['EnrollDate']).year
claim_data_per_pet_month['EnrollMonth'] = pd.DatetimeIndex(claim_data_per_pet_month['EnrollDate']).month
claim_data_per_pet_month['EnrollDay'] = pd.DatetimeIndex(claim_data_per_pet_month['EnrollDate']).day
claim_data_per_pet_month['CancelYear'] = pd.DatetimeIndex(claim_data_per_pet_month['CancelDate']).year
claim_data_per_pet_month['CancelMonth'] = pd.DatetimeIndex(claim_data_per_pet_month['CancelDate']).month
claim_data_per_pet_month['CancelDay'] = pd.DatetimeIndex(claim_data_per_pet_month['CancelDate']).day
claim_data_per_pet_month.head()

,PetId,ClaimYear,ClaimMonth,ClaimAmount,TotalClaimedAmount,ClaimsCountIntermediate,ClaimsCountRevised,InsuranceCancelled,InsurancedPeriod,ClaimFreqPerMonth,EnrollDate,CancelDate,Species,Breed,AgeAtEnroll,EnrollYear,EnrollMonth,EnrollDay,CancelYear,CancelMonth,CancelDay
0,0,0.0,0.0,0.00,0.00,1,0,0,1067,0.00000,2015-08-11,2018-07-13,Dog,Mixed Breed,5 years old,2015,8,11,2018.0,7.0,13.0
1,1,0.0,0.0,0.00,0.00,1,0,0,709,0.00000,2014-02-15,2016-01-25,Cat,Mixed Breed,3 years old,2014,2,15,2016.0,1.0,25.0
2,2,2015.0,12.0,75.32,2643.44,14,14,1,1654,0.25393,2014-12-20,NaT,Dog,French Bulldog,8 weeks to 12 months old,2014,12,20,NaN,NaN,NaN
3,2,2016.0,1.0,61.06,2643.44,14,14,1,1654,0.25393,2014-12-20,NaT,Dog,French Bulldog,8 weeks to 12 months old,2014,12,20,NaN,NaN,NaN
4,2,2017.0,6.0,63.25,2643.44,14,14,1,1654,0.25393,2014-12-20,NaT,Dog,French Bulldog,8 weeks to 12 months old,2014,12,20,NaN,NaN,NaN


Converting age to numeric

In [22]:
age_values_at_enrolled = claim_data_per_pet_month.AgeAtEnroll.unique()
print(age_values_at_enrolled)
age_converted = [5,3,0.5,1,4,7,10,9,2,8,6,11,0.08,13,12]
for i in range(len(age_values_at_enrolled)):
  claim_data_per_pet_month['AgeAtEnroll'].replace(to_replace = age_values_at_enrolled[i], value = age_converted[i] , inplace=True)
claim_data_per_pet_month.head()

['5 years old' '3 years old' '8 weeks to 12 months old' '1 year old'
 '4 years old' '7 years old' '10 years old' '9 years old' '2 years old'
 '8 years old' '6 years old' '11 years old' '0-7 weeks old' '13 years old'
 '12 years old']


,PetId,ClaimYear,ClaimMonth,ClaimAmount,TotalClaimedAmount,ClaimsCountIntermediate,ClaimsCountRevised,InsuranceCancelled,InsurancedPeriod,ClaimFreqPerMonth,EnrollDate,CancelDate,Species,Breed,AgeAtEnroll,EnrollYear,EnrollMonth,EnrollDay,CancelYear,CancelMonth,CancelDay
0,0,0.0,0.0,0.00,0.00,1,0,0,1067,0.00000,2015-08-11,2018-07-13,Dog,Mixed Breed,5.0,2015,8,11,2018.0,7.0,13.0
1,1,0.0,0.0,0.00,0.00,1,0,0,709,0.00000,2014-02-15,2016-01-25,Cat,Mixed Breed,3.0,2014,2,15,2016.0,1.0,25.0
2,2,2015.0,12.0,75.32,2643.44,14,14,1,1654,0.25393,2014-12-20,NaT,Dog,French Bulldog,0.5,2014,12,20,NaN,NaN,NaN
3,2,2016.0,1.0,61.06,2643.44,14,14,1,1654,0.25393,2014-12-20,NaT,Dog,French Bulldog,0.5,2014,12,20,NaN,NaN,NaN
4,2,2017.0,6.0,63.25,2643.44,14,14,1,1654,0.25393,2014-12-20,NaT,Dog,French Bulldog,0.5,2014,12,20,NaN,NaN,NaN


In [23]:
# process columns, apply LabelEncoder to categorical features
cols = ['Species', 'Breed']
for c in cols:
    lbl = LabelEncoder() 
    lbl.fit(list(claim_data_per_pet_month[c].values)) 
    claim_data_per_pet_month[c] = lbl.transform(list(claim_data_per_pet_month[c].values))
claim_data_per_pet_month.head()

,PetId,ClaimYear,ClaimMonth,ClaimAmount,TotalClaimedAmount,ClaimsCountIntermediate,ClaimsCountRevised,InsuranceCancelled,InsurancedPeriod,ClaimFreqPerMonth,EnrollDate,CancelDate,Species,Breed,AgeAtEnroll,EnrollYear,EnrollMonth,EnrollDay,CancelYear,CancelMonth,CancelDay
0,0,0.0,0.0,0.00,0.00,1,0,0,1067,0.00000,2015-08-11,2018-07-13,1,4,5.0,2015,8,11,2018.0,7.0,13.0
1,1,0.0,0.0,0.00,0.00,1,0,0,709,0.00000,2014-02-15,2016-01-25,0,4,3.0,2014,2,15,2016.0,1.0,25.0
2,2,2015.0,12.0,75.32,2643.44,14,14,1,1654,0.25393,2014-12-20,NaT,1,1,0.5,2014,12,20,NaN,NaN,NaN
3,2,2016.0,1.0,61.06,2643.44,14,14,1,1654,0.25393,2014-12-20,NaT,1,1,0.5,2014,12,20,NaN,NaN,NaN
4,2,2017.0,6.0,63.25,2643.44,14,14,1,1654,0.25393,2014-12-20,NaT,1,1,0.5,2014,12,20,NaN,NaN,NaN


# Feature Selection

In [24]:
total_features = ['Species', 'Breed', 'AgeAtEnroll', 'EnrollYear',	'EnrollMonth', 'EnrollDay',	'CancelYear',	'CancelMonth',	'CancelDay',	'ClaimYear',	'ClaimMonth',  'TotalClaimedAmount', 'ClaimAmount', 'ClaimsCountRevised', 'InsuranceCancelled', 'InsurancedPeriod' , 'ClaimFreqPerMonth' ]
selected_features = ['Species', 'Breed', 'AgeAtEnroll', 'EnrollYear',	'EnrollMonth', 'EnrollDay',	'CancelYear',	'CancelMonth',	'CancelDay',	'ClaimYear',	'ClaimMonth',  'TotalClaimedAmount', 'ClaimsCountRevised', 'InsuranceCancelled', 'InsurancedPeriod' , 'ClaimFreqPerMonth' ]
selected_features.append('ClaimAmount')
final_features_selected = claim_data_per_pet_month[selected_features]
final_features_selected.head()

,Species,Breed,AgeAtEnroll,EnrollYear,EnrollMonth,EnrollDay,CancelYear,CancelMonth,CancelDay,ClaimYear,ClaimMonth,TotalClaimedAmount,ClaimsCountRevised,InsuranceCancelled,InsurancedPeriod,ClaimFreqPerMonth,ClaimAmount
0,1,4,5.0,2015,8,11,2018.0,7.0,13.0,0.0,0.0,0.00,0,0,1067,0.00000,0.00
1,0,4,3.0,2014,2,15,2016.0,1.0,25.0,0.0,0.0,0.00,0,0,709,0.00000,0.00
2,1,1,0.5,2014,12,20,NaN,NaN,NaN,2015.0,12.0,2643.44,14,1,1654,0.25393,75.32
3,1,1,0.5,2014,12,20,NaN,NaN,NaN,2016.0,1.0,2643.44,14,1,1654,0.25393,61.06
4,1,1,0.5,2014,12,20,NaN,NaN,NaN,2017.0,6.0,2643.44,14,1,1654,0.25393,63.25


In [25]:
final_features_selected_zero = final_features_selected.fillna(0)
final_features_selected_None = final_features_selected.fillna('None')
final_features_selected_zero.head()

,Species,Breed,AgeAtEnroll,EnrollYear,EnrollMonth,EnrollDay,CancelYear,CancelMonth,CancelDay,ClaimYear,ClaimMonth,TotalClaimedAmount,ClaimsCountRevised,InsuranceCancelled,InsurancedPeriod,ClaimFreqPerMonth,ClaimAmount
0,1,4,5.0,2015,8,11,2018.0,7.0,13.0,0.0,0.0,0.00,0,0,1067,0.00000,0.00
1,0,4,3.0,2014,2,15,2016.0,1.0,25.0,0.0,0.0,0.00,0,0,709,0.00000,0.00
2,1,1,0.5,2014,12,20,0.0,0.0,0.0,2015.0,12.0,2643.44,14,1,1654,0.25393,75.32
3,1,1,0.5,2014,12,20,0.0,0.0,0.0,2016.0,1.0,2643.44,14,1,1654,0.25393,61.06
4,1,1,0.5,2014,12,20,0.0,0.0,0.0,2017.0,6.0,2643.44,14,1,1654,0.25393,63.25


In [26]:
final_features_selected.describe()

,Species,Breed,AgeAtEnroll,EnrollYear,EnrollMonth,EnrollDay,CancelYear,CancelMonth,CancelDay,ClaimYear,ClaimMonth,TotalClaimedAmount,ClaimsCountRevised,InsuranceCancelled,InsurancedPeriod,ClaimFreqPerMonth,ClaimAmount
count,28673.000000,28673.000000,28673.000000,28673.000000,28673.000000,28673.000000,10709.000000,10709.000000,10709.000000,28673.000000,28673.000000,28673.000000,28673.000000,28673.000000,28673.000000,28607.000000,28673.000000
mean,0.738604,3.394169,2.649227,2013.696369,6.518502,15.795069,2016.680549,6.651135,16.195443,1596.144282,5.040073,7571.958381,24.667213,0.626513,1624.124821,0.401061,525.899504
std,0.439403,1.189072,3.180400,2.438248,3.419466,8.563119,2.109758,3.349611,8.841547,818.774254,3.985539,11524.284745,35.515293,0.483738,942.744013,0.541199,1381.544131
min,0.000000,0.000000,0.080000,2009.000000,1.000000,1.000000,2010.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,4.000000,0.500000,2012.000000,4.000000,8.000000,2015.000000,4.000000,9.000000,2012.000000,1.000000,330.970000,2.000000,0.000000,881.000000,0.042336,30.760000
50%,1.000000,4.000000,0.500000,2014.000000,6.000000,16.000000,2017.000000,7.000000,17.000000,2016.000000,5.000000,3266.820000,11.000000,1.000000,1624.000000,0.209860,138.600000
75%,1.000000,4.000000,4.000000,2015.000000,9.000000,23.000000,2018.000000,10.000000,24.000000,2018.000000,8.000000,10036.900000,34.000000,1.000000,2337.000000,0.566229,425.650000
max,1.000000,5.000000,13.000000,2019.000000,12.000000,31.000000,2019.000000,12.000000,31.000000,2019.000000,12.000000,110202.380000,306.000000,1.000000,3610.000000,15.000000,48691.460000


In [27]:
cols = ['CancelYear', 'CancelMonth', 'CancelDay', 'ClaimYear', 'ClaimMonth' ]
for i in cols:
  final_features_selected_zero[i] = final_features_selected_zero[i].astype(int)
final_features_selected_zero.head()

,Species,Breed,AgeAtEnroll,EnrollYear,EnrollMonth,EnrollDay,CancelYear,CancelMonth,CancelDay,ClaimYear,ClaimMonth,TotalClaimedAmount,ClaimsCountRevised,InsuranceCancelled,InsurancedPeriod,ClaimFreqPerMonth,ClaimAmount
0,1,4,5.0,2015,8,11,2018,7,13,0,0,0.00,0,0,1067,0.00000,0.00
1,0,4,3.0,2014,2,15,2016,1,25,0,0,0.00,0,0,709,0.00000,0.00
2,1,1,0.5,2014,12,20,0,0,0,2015,12,2643.44,14,1,1654,0.25393,75.32
3,1,1,0.5,2014,12,20,0,0,0,2016,1,2643.44,14,1,1654,0.25393,61.06
4,1,1,0.5,2014,12,20,0,0,0,2017,6,2643.44,14,1,1654,0.25393,63.25


Feature and Label 

In [28]:
y =final_features_selected_zero['ClaimAmount']
X = final_features_selected_zero.drop(['ClaimAmount'], axis=1)
y.describe()

count    28673.000000
mean       525.899504
std       1381.544131
min          0.000000
25%         30.760000
50%        138.600000
75%        425.650000
max      48691.460000
Name: ClaimAmount, dtype: float64

In [0]:
#train test split 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [0]:
# Normalizing the dataset to make sure all the data is in range of 0 to 1 
scaler = MinMaxScaler(feature_range=(0, 1))
X_train_rescaled = scaler.fit_transform(X_train)
X_test_rescaled = scaler.fit_transform(X_test)

In [31]:
model = xgb.XGBRegressor()
model.fit(X_train_rescaled, y_train)
ypred_xgb_reg_pred = model.predict(X_test_rescaled)
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, ypred_xgb_reg_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, ypred_xgb_reg_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, ypred_xgb_reg_pred)))

/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[21:15:25] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Mean Absolute Error: 476.74816120046887
Mean Squared Error: 1408188.7277662978
Root Mean Squared Error: 1186.6712804169053


In [32]:
destree = tree.DecisionTreeRegressor()
destree.fit(X_train_rescaled, y_train)
ydecpredTree = destree.predict(X_test_rescaled)
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, ydecpredTree))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, ydecpredTree))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, ydecpredTree)))

Mean Absolute Error: 684.5845963382737
Mean Squared Error: 3295528.2642753613
Root Mean Squared Error: 1815.3589904686514


In [33]:
randomForest = RandomForestRegressor()
randomForest.fit(X_train_rescaled, y_train)
ydecpredRand = randomForest.predict(X_test_rescaled)
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, ydecpredRand))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, ydecpredRand))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, ydecpredRand)))

Mean Absolute Error: 536.3472350828247
Mean Squared Error: 1663326.6484503415
Root Mean Squared Error: 1289.7002165039523


In [34]:
gradBoost = GradientBoostingRegressor()
gradBoost.fit(X_train_rescaled, y_train)
ydecpredRand = gradBoost.predict(X_test_rescaled)
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, ydecpredRand))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, ydecpredRand))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, ydecpredRand)))


Mean Absolute Error: 476.2877369598796
Mean Squared Error: 1400313.6675113093
Root Mean Squared Error: 1183.3484979123052


# Test data preprocessing

In [35]:
pet_data_test = pd.merge(pet_data_df, extracted_features, how='inner' , on = 'PetId')
age_values_at_enrolled = pet_data_test.AgeAtEnroll.unique()
age_converted = [5,3,0.5,1,4,7,10,9,2,8,6,11,0.08,13,12]
for i in range(len(age_values_at_enrolled)):
  pet_data_test['AgeAtEnroll'].replace(to_replace = age_values_at_enrolled[i], value = age_converted[i] , inplace=True)
pet_data_test.head()

,PetId,EnrollDate,CancelDate,Species,Breed,AgeAtEnroll,TotalClaimedAmount,ClaimsCountIntermediate,ClaimsCountRevised,InsuranceCancelled,InsurancedPeriod,ClaimFreqPerMonth
0,0,2015-08-11,2018-07-13,Dog,Mixed Breed,5.0,0.00,1,0,0,1067,0.000000
1,1,2014-02-15,2016-01-25,Cat,Mixed Breed,3.0,0.00,1,0,0,709,0.000000
2,2,2014-12-20,NaT,Dog,French Bulldog,0.5,2643.44,14,14,1,1654,0.253930
3,3,2015-01-09,2015-01-14,Dog,Mixed Breed,1.0,0.00,1,0,0,5,0.000000
4,4,2014-07-20,NaT,Cat,Mixed Breed,0.5,1356.79,1,1,1,1807,0.016602


In [0]:
pet_data_test['EnrollYear'] = pd.DatetimeIndex(pet_data_test['EnrollDate']).year
pet_data_test['EnrollMonth'] = pd.DatetimeIndex(pet_data_test['EnrollDate']).month
pet_data_test['EnrollDay'] = pd.DatetimeIndex(pet_data_test['EnrollDate']).day
pet_data_test['CancelYear'] = pd.DatetimeIndex(pet_data_test['CancelDate']).year
pet_data_test['CancelMonth'] = pd.DatetimeIndex(pet_data_test['CancelDate']).month
pet_data_test['CancelDay'] = pd.DatetimeIndex(pet_data_test['CancelDate']).day
pet_data_test['ClaimYear'] = '2019'
pet_data_test['ClaimMonth'] = '07'

# process columns, apply LabelEncoder to categorical features
cols = ['Species', 'Breed']
for c in cols:
    lbl = LabelEncoder() 
    lbl.fit(list(pet_data_test[c].values)) 
    pet_data_test[c] = lbl.transform(list(pet_data_test[c].values))
pet_data_test.head()
petId = pet_data_test['PetId']
selected_features.remove('ClaimAmount')
pet_data_test_final = pet_data_test[selected_features]
pet_data_test_final = pet_data_test_final.fillna(0)
pet_data_test_final['CancelYear'] = pet_data_test_final['CancelYear'].astype(int)
pet_data_test_final['CancelMonth'] = pet_data_test_final['CancelMonth'].astype(int)
pet_data_test_final['CancelDay'] = pet_data_test_final['CancelDay'].astype(int)
pet_data_test_final['ClaimYear'] = pet_data_test_final['ClaimYear'].astype(int)
pet_data_test_final['ClaimMonth'] = pet_data_test_final['ClaimMonth'].astype(int)

pet_data_test_final.head()
# Normalizing the dataset to make sure all the data is in range of 0 to 1 
scaler = MinMaxScaler(feature_range=(0, 1))
X_rescaled = scaler.fit_transform(X)
pet_data_test_final_scaled = scaler.fit_transform(pet_data_test_final)

In [0]:
from sklearn.ensemble import GradientBoostingRegressor
gradBoost = GradientBoostingRegressor()
gradBoost.fit(X_rescaled, y)
ypred_gradBoost = gradBoost.predict(pet_data_test_final_scaled)

In [0]:
submission = pd.DataFrame({
        "petId": petId,
        "CliamAmount": ypred_gradBoost
    })
from google.colab import files
submission.to_csv('Pet_claim_predicted_july.csv', index=False)